In [1]:
import numpy as np
from numba import jit
from time import time_ns

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd

@jit
def exp(a:float):
    return np.exp(a)

class State():
    beta = 10
    def __init__(self, size:int=10):
        self.size = size
        self.value = np.array([False] * size)
        self.inter = np.array([[float(0)]*size]*size)
        self.field = np.array([float(0)]*size)
        self.flip_pro = np.array([float(0)]*size)
        self.N = 1
        self.goal = np.array([[False]*size]*self.N)

    def __str__(self):
        return ("Values\n "+str(list(map(int, self.value)))
        +"\nInteractions\n"+str(self.inter)+"\nField\n "+str(self.field))

    def setInterByIndex(self, i:int, j:int, value:float):
        try:
            self.inter[i][j] = value
        except IndexError:
            print("Index Out of Range: "+ str(i)+', '+str(j))
        return value
    def setInter(self, li: np.array):
        if len(li) != self.size or len(li[0])!= self.size:
            print("SetInter Data Size Error")
            return
        self.inter = li
        return
    def setValue(self, li: np.array):
        if len(li) != self.size:
            print("SetValues Data Size Error")
            return
        self.value = li
        return
    def setField(self, li: np.array):
        if len(li) != self.size:
            print("SetField Data Size Error")
            return
        self.field = li
        return
    def hamiltonian(self):
        return sum(sum(pow((self.value*self.inter).T*self.value, 1))) + sum(self.value*self.field)
    def roll(self):
        self.value = np.random.randint(0, 2, size=self.size, dtype=bool)


    def calcField(self):
        ans = list()
        for i in range(self.size):
            ans.append((self.inter[i]*self.value).sum())
        self.field = np.array(ans)
        return self.field

    def calcFlip(self):
        Del_H = self.field - self.field*self.value*2
        self.flip_pro = np.exp(-self.beta*Del_H)
        self.flip_pro = self.flip_pro / self.flip_pro.sum()
        # if Del_H<0: self.flip_pro=1
        # else: self.flip_pro=0
        return self.flip_pro

    def updateField(self, index:int):
        
        return self.field

    def step(self):
        # self.calcField() # N^2 Replaced with updatefield
        start = (time_ns())
        n = np.random.randint(0, self.size)
        val = self.value[n]
        field = self.field[n]
        # print(1, time_ns()-start)
        if (val and field<0) or ((not val) and field>0):
            self.value[n] = not self.value[n]
            # print(2, time_ns()-start)
            self.field += self.inter.T[n] * (2-4*self.value[n])
            # print(3, time_ns()-start)
        else:
            p = np.exp(-self.beta*field*(val*2-1))
            # print(4, time_ns()-start)
            
            if np.random.uniform() < p:
                self.value[n] = not self.value[n]
                self.field += self.inter.T[n] * (2-4*self.value[n])
            # print(5, time_ns()-start)
        return self.value

    def setGoal(self, goal:np.ndarray, option="hopfield"):
        self.N = len(goal)
        self.goal = goal
        if option == "hopfield":
            self.hopfield_interaction()
        return self.inter

    def hopfield_interaction(self):
        self.field = np.array([float(0)]*self.size)
        self.inter += np.dot((self.goal*2-1).T, (self.goal*2-1))/self.N
    
    def eval_interaction(self, flip_number=10, steps=1000):
        dataset=self.goal
        modified = [flip(data, flip_number=flip_number) for data in dataset]
        result = []
        for (index, start) in enumerate(modified):
            self.setValue(start)
            for _ in range(steps):
                if(_%100==0): self.calcField()
                self.step()
            result.append((self.value^dataset[index]).sum())
        return np.array(result).mean()
        
    def small_adj(self, learning_rate=0.1, delta=0.1):
        loss0 = self.eval_interaction()
        loss_difference = np.array([[float(0)]*self.size]*self.size)
        for i in range(self.size):
            for j in range(self.size):
                self.inter[i][j] += delta
                loss1 = self.eval_interaction()
                self.inter[i][j] -= delta
                loss_difference[i][j] = (loss1-loss0)/delta
        self.inter -= loss_difference * learning_rate
        return self.eval_interaction()

    def plot_loss(self, flip_number=10, steps=1000, count=100):
        data = [self.eval_interaction(flip_number=flip_number, steps=steps) for _ in range(count)]
        # sns.histplot(
        #     data
        # )
        return data

def flip(input: np.ndarray, flip_number:int) :
    arr = input.copy()
    for index in np.random.randint(0, len(arr), dtype=int, size=flip_number):
        arr[index] = not arr[index]
    return arr


In [2]:
size = 200; modification = 10
inter = np.array([[float(0)]*size]*size)
dataset=np.array([np.random.randint(0, 2, size=size, dtype=bool) for _ in range(20)])
state = State(size)
state.setGoal(dataset)
rand = np.random.uniform

answer_inter = state.inter

damage_node = pd.DataFrame()
for n in range(201):
    temp = answer_inter
    for i in np.random.choice(size, n):
        for j in range(size):
            temp[i][j] = 0
            temp[j][i] = 0
    state.setInter(temp)
    damage_node[n]=state.plot_loss()
    sns.histplot(
        damage_node[n]
    ).get_figure().savefig(f"damaged_node/node{str(n)}.png")
    plt.close()
    damage_node.to_csv(f"damaged_node/damaged_node.csv")
    print(n)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


<ipython-input-2-daeb46896a1a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_node[n]=state.plot_loss()


100


<ipython-input-2-daeb46896a1a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_node[n]=state.plot_loss()


101


...

<ipython-input-2-daeb46896a1a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_node[n]=state.plot_loss()


198


<ipython-input-2-daeb46896a1a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_node[n]=state.plot_loss()


199
200


<ipython-input-2-daeb46896a1a>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  damage_node[n]=state.plot_loss()
